# Modeling and Simulation in Python

Chapter 9

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [1]:
try:
    import pint
except ImportError:
    !pip install pint
    import pint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 1.2 MB/s eta 0:00:0000:0100:01


In [2]:
try:
    from modsim import *
except ImportError:
    !pip install modsimpy
    from modsim import *

  Using cached modsimpy-1.1.3.tar.gz (19 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for modsimpy: filename=modsimpy-1.1.3-py3-none-any.whl size=20338 sha256=27f88013c6f6755ed94a87391fc0776fc6e36b59c9ce70a04f43cc9e16b25236
  Stored in directory: /Users/waheebalgabri/Library/Caches/pip/wheels/81/a2/77/2b368ef9c265ce05265cf849ebde2785419b17a3db91ada9ff
Successfully built modsimpy


In [3]:
# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import everything from SymPy.
from sympy import *

# Set up Jupyter notebook to display math.
init_printing()

The following displays SymPy expressions and provides the option of showing results in LaTeX format.

In [42]:
from sympy.printing import latex

def show(expr, show_latex=False):
    """Display a SymPy expression.

    expr: SymPy expression
    show_latex: boolean
    """
    if show_latex:
        print(latex(expr))
    return expr

### Analysis with SymPy

Create a symbol for time.

In [43]:
t = symbols('t')

t

If you combine symbols and numbers, you get symbolic expressions.

In [44]:
expr = t + 1

t + 1

The result is an `Add` object, which just represents the sum without trying to compute it.

In [45]:
type(expr)

sympy.core.add.Add

`subs` can be used to replace a symbol with a number, which allows the addition to proceed.

In [46]:
expr.subs(t, 2)

3

`f` is a special class of symbol that represents a function.

In [47]:
f = Function('f')

f

The type of `f` is `UndefinedFunction`

In [48]:
type(f)

sympy.core.function.UndefinedFunction

SymPy understands that `f(t)` means `f` evaluated at `t`, but it doesn't try to evaluate it yet.

In [49]:
f(t)

f(t)

`diff` returns a `Derivative` object that represents the time derivative of `f`

In [54]:
dfdt = diff(f(t), t)

d       
──(f(t))
dt      

In [55]:
type(dfdt)

sympy.core.function.Derivative

We need a symbol for `alpha`

In [56]:
alpha = symbols('alpha')

α

Now we can write the differential equation for proportional growth.

In [57]:
eq1 = Eq(dfdt, alpha*f(t))

d                
──(f(t)) = α⋅f(t)
dt               

And use `dsolve` to solve it.  The result is the general solution.

In [58]:
solution_eq = dsolve(eq1)

           α⋅t
f(t) = C₁⋅ℯ   

We can tell it's a general solution because it contains an unspecified constant, `C1`.

In this example, finding the particular solution is easy: we just replace `C1` with `p_0`

In [61]:
C1, p_0 = symbols('C1 p_0')

In [62]:
particular = solution_eq.subs(C1, p_0)

           α⋅t
f(t) = p₀⋅ℯ   

In the next example, we have to work a little harder to find the particular solution.

### Solving the quadratic growth equation

We'll use the (r, K) parameterization, so we'll need two more symbols:

In [63]:
r, K = symbols('r K')

Now we can write the differential equation.

In [64]:
eq2 = Eq(diff(f(t), t), r * f(t) * (1 - f(t)/K))

d            ⎛    f(t)⎞     
──(f(t)) = r⋅⎜1 - ────⎟⋅f(t)
dt           ⎝     K  ⎠     

And solve it.

In [65]:
solution_eq = dsolve(eq2)

           C₁⋅K + r⋅t 
        K⋅ℯ           
f(t) = ───────────────
        C₁⋅K + r⋅t    
       ℯ           - 1

The result, `solution_eq`, contains `rhs`, which is the right-hand side of the solution.

In [66]:
general = solution_eq.rhs

    C₁⋅K + r⋅t 
 K⋅ℯ           
───────────────
 C₁⋅K + r⋅t    
ℯ           - 1

We can evaluate the right-hand side at $t=0$

In [25]:
at_0 = general.subs(t, 0)

    C₁⋅K 
 K⋅ℯ     
─────────
 C₁⋅K    
ℯ     - 1

Now we want to find the value of `C1` that makes `f(0) = p_0`.

So we'll create the equation `at_0 = p_0` and solve for `C1`.  Because this is just an algebraic identity, not a differential equation, we use `solve`, not `dsolve`.

The result from `solve` is a list of solutions.  In this case, [we have reason to expect only one solution](https://en.wikipedia.org/wiki/Picard%E2%80%93Lindel%C3%B6f_theorem), but we still get a list, so we have to use the bracket operator, `[0]`, to select the first one.

In [26]:
solutions = solve(Eq(at_0, p_0), C1)
type(solutions), len(solutions)

(list, 1)

In [27]:
value_of_C1 = solutions[0]

   ⎛ -p₀  ⎞
log⎜──────⎟
   ⎝K - p₀⎠
───────────
     K     

Now in the general solution, we want to replace `C1` with the value of `C1` we just figured out.

In [28]:
particular = general.subs(C1, value_of_C1)

             r⋅t        
      -K⋅p₀⋅ℯ           
────────────────────────
         ⎛      r⋅t    ⎞
         ⎜  p₀⋅ℯ       ⎟
(K - p₀)⋅⎜- ─────── - 1⎟
         ⎝   K - p₀    ⎠

The result is complicated, but SymPy provides a method that tries to simplify it.

In [29]:
particular = simplify(particular)

         r⋅t    
   K⋅p₀⋅ℯ       
────────────────
        r⋅t     
K + p₀⋅ℯ    - p₀

Often simplicity is in the eye of the beholder, but that's about as simple as this expression gets.

Just to double-check, we can evaluate it at `t=0` and confirm that we get `p_0`

In [30]:
particular.subs(t, 0)

p₀

This solution is called the [logistic function](https://en.wikipedia.org/wiki/Population_growth#Logistic_equation).

In some places you'll see it written in a different form:

$f(t) = \frac{K}{1 + A e^{-rt}}$

where $A = (K - p_0) / p_0$.

We can use SymPy to confirm that these two forms are equivalent.  First we represent the alternative version of the logistic function:

In [31]:
A = (K - p_0) / p_0

K - p₀
──────
  p₀  

In [32]:
logistic = K / (1 + A * exp(-r*t))

        K         
──────────────────
              -r⋅t
    (K - p₀)⋅ℯ    
1 + ──────────────
          p₀      

To see whether two expressions are equivalent, we can check whether their difference simplifies to 0.

In [33]:
simplify(particular - logistic)

0

This test only works one way: if SymPy says the difference reduces to 0, the expressions are definitely equivalent (and not just numerically close).

But if SymPy can't find a way to simplify the result to 0, that doesn't necessarily mean there isn't one.  Testing whether two expressions are equivalent is a surprisingly hard problem; in fact, there is no algorithm that can solve it in general.

### Exercises

**Exercise:** Solve the quadratic growth equation using the alternative parameterization

$\frac{df(t)}{dt} = \alpha f(t) + \beta f^2(t) $

In [67]:
# creating the symbols
alpha, beta = symbols('alpha beta')

In [68]:
# constructing equation
eq3 = Eq(diff(f(t), t), alpha*f(t) + beta*f(t)**2)

d                      2   
──(f(t)) = α⋅f(t) + β⋅f (t)
dt                         

In [69]:
# solving equation
solution_eq = dsolve(eq3)

             α⋅(C₁ + t)   
          α⋅ℯ             
f(t) = ───────────────────
         ⎛     α⋅(C₁ + t)⎞
       β⋅⎝1 - ℯ          ⎠

In [70]:
# general solution
general = solution_eq.rhs

      α⋅(C₁ + t)   
   α⋅ℯ             
───────────────────
  ⎛     α⋅(C₁ + t)⎞
β⋅⎝1 - ℯ          ⎠

In [71]:
# Solution at t=0
at_0 = general.subs(t, 0)

      C₁⋅α   
   α⋅ℯ       
─────────────
  ⎛     C₁⋅α⎞
β⋅⎝1 - ℯ    ⎠

In [72]:
# Finding value of C1
solutions = solve(Eq(at_0, p_0), C1)
value_of_C1 = solutions[0]

   ⎛  β⋅p₀  ⎞
log⎜────────⎟
   ⎝α + β⋅p₀⎠
─────────────
      α      

In [73]:
#evaluate at t=0 and confirm that we get p_0
particular = general.subs(C1, value_of_C1)
particular.simplify()

           α⋅t      
     α⋅p₀⋅ℯ         
────────────────────
          α⋅t       
α - β⋅p₀⋅ℯ    + β⋅p₀

**Exercise:**  Use [WolframAlpha](https://www.wolframalpha.com/) to solve the quadratic growth model, using either or both forms of parameterization:

    df(t) / dt = alpha f(t) + beta f(t)^2

or

    df(t) / dt = r f(t) (1 - f(t)/K)

Find the general solution and also the particular solution where `f(0) = p_0`.

In [8]:
import os 

![unknown.gif](unknown.gif)